# Managing Kleio files

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [6]:
# NBVAL_IGNORE_OUTPUT

from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook(
    kleio_version='12.6.576')
tlnb.print_info(show_token=True)


Timelink version: 1.1.14
Project name: dehergne-locations
Project home: /Users/jrc/mhk-home/sources/dehergne-locations
Database type: sqlite
Database name: test.db
Kleio image: timelinkserver/kleio-server
Kleio server token: jLZtIeWrMF17H5GoLXz1n4Sf1Gt6EKYf
Kleio server URL: http://127.0.0.1:8092
Kleio server home: /Users/jrc/mhk-home/sources/dehergne-locations
Kleio server container: dreamy_ramanujan
Kleio version requested: 12.6.576
Kleio server version: 12.6.576 (2024-09-09 17:57:04)
SQLite directory: /Users/jrc/develop/timelink-py/tests/sqlite
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne-locations, project_home=/Users/jrc/mhk-home/sources/dehergne-locations, db_type=sqlite, db_name=test.db, kleio_image=timelinkserver/kleio-server, kleio_version=12.6.576, postgres_image=postgres, postgres_version=latest)


TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

### Database status

Count the number of rows in each table in the database.


In [7]:
# NBVAL_IGNORE_OUTPUT

tlnb.table_row_count_df()

,table,count
0,acts,1
1,aregisters,0
2,attributes,348
3,class_attributes,63
4,classes,13
5,entities,450
6,geoentities,0
7,goods,0
8,kleiofiles,1
9,links,0


### Show the kleio files available 

In [8]:
# NBVAL_IGNORE_OUTPUT

kleio_files = tlnb.get_kleio_files()
kleio_files

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,sources/dehergne-0-abrev.cli,dehergne-0-abrev.cli,2024-05-30 11:15:26.243149+00:00,V,2024-05-30 11:15:00+00:00,0,0,N,0,0,None,None,None,/rest/reports/sources/dehergne-0-abrev.rpt,/rest/exports/sources/dehergne-0-abrev.xml
1,sources/dehergne-a.cli,dehergne-a.cli,2024-08-28 09:38:42.920744+00:00,V,2024-08-28 09:38:00+00:00,0,0,N,0,0,None,None,None,/rest/reports/sources/dehergne-a.rpt,/rest/exports/sources/dehergne-a.xml
2,sources/dehergne-b.cli,dehergne-b.cli,2024-06-03 01:55:04.420127+00:00,V,2024-06-02 14:17:00+00:00,0,0,N,0,0,None,None,None,/rest/reports/sources/dehergne-b.rpt,/rest/exports/sources/dehergne-b.xml
3,sources/dehergne-c.cli,dehergne-c.cli,2024-08-29 09:51:59.978483+00:00,V,2024-08-29 09:52:00+00:00,0,0,N,0,0,None,None,None,/rest/reports/sources/dehergne-c.rpt,/rest/exports/sources/dehergne-c.xml
4,sources/dehergne-d.cli,dehergne-d.cli,2024-07-08 04:03:56.544803+00:00,V,2024-07-08 04:03:00+00:00,0,0,N,0,0,None,None,None,/rest/reports/sources/dehergne-d.rpt,/rest/exports/sources/dehergne-d.xml
5,sources/dehergne-e.cli,dehergne-e.cli,2024-05-30 11:15:43.310451+00:00,V,2024-05-30 11:15:00+00:00,0,0,N,0,0,None,None,None,/rest/reports/sources/dehergne-e.rpt,/rest/exports/sources/dehergne-e.xml
6,sources/dehergne-f.cli,dehergne-f.cli,2024-08-28 09:31:07.362158+00:00,V,2024-08-28 09:31:00+00:00,0,0,N,0,0,None,None,None,/rest/reports/sources/dehergne-f.rpt,/rest/exports/sources/dehergne-f.xml
7,sources/dehergne-g.cli,dehergne-g.cli,2024-06-21 06:52:04.080442+00:00,V,2024-06-11 14:11:00+00:00,0,0,N,0,0,None,None,None,/rest/reports/sources/dehergne-g.rpt,/rest/exports/sources/dehergne-g.xml
8,sources/dehergne-h.cli,dehergne-h.cli,2024-05-30 11:15:50.972075+00:00,V,2024-05-30 11:15:00+00:00,0,0,N,0,0,None,None,None,/rest/reports/sources/dehergne-h.rpt,/rest/exports/sources/dehergne-h.xml
9,sources/dehergne-i.cli,dehergne-i.cli,2024-05-31 04:00:05.521890+00:00,V,2024-05-31 04:00:00+00:00,0,0,N,0,0,None,None,None,/rest/reports/sources/dehergne-i.rpt,/rest/exports/sources/dehergne-i.xml


## Optionally clean translations 

In [45]:
# NBVAL_IGNORE_OUTPUT

# tlnb.kleio_server.translation_clean("", recurse="yes")


# Update the database from Kleio sources.

Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [9]:
# NBVAL_IGNORE_OUTPUT

import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources(path='', with_import_errors=True)
# get the import status
import_status = tlnb.get_import_status()
# list those with import_error > 0 or error > 0
errors = import_status.query("import_errors > 0 or errors > 0")
if len(errors) > 0:
    for error in errors.iterrows():
        print(error)

INFO:root:Importing sources/dehergne-0-abrev.cli
INFO:root:Importing sources/dehergne-a.cli
INFO:root:Storing 14 postponed relations
INFO:root:Importing sources/dehergne-b.cli
INFO:root:Storing 28 postponed relations
INFO:root:Importing sources/dehergne-c.cli
INFO:root:Storing 24 postponed relations
INFO:root:Importing sources/dehergne-d.cli
INFO:root:Storing 13 postponed relations
INFO:root:Importing sources/dehergne-e.cli
INFO:root:Storing 1 postponed relations
INFO:root:Importing sources/dehergne-f.cli
INFO:root:Storing 18 postponed relations
INFO:root:Importing sources/dehergne-g.cli
INFO:root:Storing 31 postponed relations
INFO:root:Importing sources/dehergne-h.cli
INFO:root:Storing 4 postponed relations
INFO:root:Importing sources/dehergne-i.cli
INFO:root:Importing sources/dehergne-j.cli
INFO:root:Storing 1 postponed relations
INFO:root:Importing sources/dehergne-k.cli
INFO:root:Storing 9 postponed relations
INFO:root:Importing sources/dehergne-l.cli
INFO:root:Storing 26 postpone

(1, path                                             sources/dehergne-a.cli
name                                                     dehergne-a.cli
size                                                             126328
directory                                                       sources
modified                               2024-08-28 09:38:42.920744+00:00
modified_iso                                  2024-08-28 09:38:42+00:00
modified_string                                     2024-08-28 09:38:42
qtime                                         1970-01-01 00:00:00+00:00
qtime_string                                        1970-01-01 00:00:00
source_url                         /rest/sources/sources/dehergne-a.cli
status                                                                V
translated                                    2024-08-28 09:38:00+00:00
translated_string                             2024-08-28T09:38:00+00:00
errors                                                      

## Check the status of the files

Check the import status of the translated files

* I: Imported
* E: Imported with error
* W: Imported with warnings no errors
* N: Not imported
* U: Translation updated need to reimport

In [47]:
# NBVAL_IGNORE_OUTPUT

imported_files_df = tlnb.get_import_status()
imported_files_df[["import_status","import_errors","import_warnings","name","errors","warnings","path"]].sort_values("name")


,import_status,import_errors,import_warnings,name,errors,warnings,path
0,I,0,0,Devedores.cli,0,0,sources/reference_sources/varia/Devedores.cli
1,I,0,0,auc-alunos.cli,0,0,sources/reference_sources/varia/auc-alunos.cli
2,I,0,0,auc_cartulario18.cli,0,0,sources/reference_sources/varia/auc_cartulario...
3,I,0,0,avexplorfa.cli,0,0,sources/reference_sources/varia/avexplorfa.cli
4,I,0,0,bap-com-celebrantes.cli,0,0,sources/reference_sources/paroquiais/baptismos...
5,I,0,0,bapt1714.cli,0,0,sources/reference_sources/paroquiais/baptismos...
6,I,0,0,bapteirasproblem1.cli,0,0,sources/reference_sources/paroquiais/baptismos...
7,I,0,0,bapteirasproblem2.cli,0,0,sources/reference_sources/paroquiais/baptismos...
8,I,0,0,cas1714-1722-com-celebrante.cli,0,0,sources/reference_sources/paroquiais/casamento...
9,I,0,0,cas1714-1722.cli,0,0,sources/reference_sources/paroquiais/casamento...


In [48]:
errors = imported_files_df[imported_files_df.errors>0][["status", "errors","import_warnings","name","imported","warnings","path"]].sort_values("name")
errors

,status,errors,import_warnings,name,imported,warnings,path


### Check a translation report

If needed to check for errors.


In [49]:
# NBVAL_IGNORE_OUTPUT

file_number=list(errors.index.unique())
if len(file_number) > 0:
    rpt = tlnb.get_translation_report(imported_files_df, file_number)
    print(rpt)
else:
    print("No errors found")



No errors found


### Get the import report for a file

In case there are errors in the import phase check the import report.

In [50]:
# NBVAL_IGNORE_OUTPUT

ierrors = imported_files_df[imported_files_df.import_errors>0][["import_status", "import_errors","import_warnings","name","imported","errors","warnings","path"]].sort_values("name")
list(ierrors.index)

[]

In [51]:

rpt = tlnb.get_import_rpt(imported_files_df, rows=list(ierrors.index))
print(rpt)